In [ ]:
import pandas as pd
import re

# Semantic Filtering

In [ ]:
df_full_news = pd.read_csv('allNews2022.csv')
df_full_news

,region,date,news
0,belgorod,2022-01-01,Центр
1,belgorod,2022-01-01,Приволжье
2,belgorod,2022-01-01,Сибирь
3,belgorod,2022-01-01,Урал
4,belgorod,2022-01-01,Юг
...,...,...,...
3014916,grozniy,2022-12-31,Глава ЧР прошелся по центральной площади Грозного
3014917,grozniy,2022-12-31,Р. Кадыров поздравил Информационное управление...
3014918,grozniy,2022-12-31,М. Хучиев награжден орденом «За заслуги перед ...
3014919,grozniy,2022-12-31,В 2022 году в Чеченской Республике отремонтиро...


In [ ]:
to_delete = ['Центр', 'Приволжье', 'Сибирь', 'Урал', 'Юг', 'Северо-Запад', 'Дальний Восток', 'Северный Кавказ']

In [ ]:
def delete_errors(data:pd.DataFrame, del_list:list):
  pattern = '|'.join(del_list)
  pattern2 = f'^({pattern})$'
  m = data['news'].str.fullmatch(pattern2, case=False, na=False)
  data = data[~m]
  return data

In [ ]:
df_full_news = delete_errors(df_full_news, to_delete)
df_full_news

,region,date,news
8,belgorod,2022-01-01,Белгородцы пробежали 2022 метра в новогоднем з...
9,belgorod,2022-01-01,Рифат Сабитов присоединился к акции «Ёлка жела...
10,belgorod,2022-01-01,В крытом футбольном манеже Белгорода прошла пе...
11,belgorod,2022-01-01,Новогодняя ночь в Белгородской области прошла ...
12,belgorod,2022-01-01,Вячеслав Гладков: к строительству детских площ...
...,...,...,...
3014916,grozniy,2022-12-31,Глава ЧР прошелся по центральной площади Грозного
3014917,grozniy,2022-12-31,Р. Кадыров поздравил Информационное управление...
3014918,grozniy,2022-12-31,М. Хучиев награжден орденом «За заслуги перед ...
3014919,grozniy,2022-12-31,В 2022 году в Чеченской Республике отремонтиро...


In [ ]:
anchor_texts = [
    "социально-экономическое развитие региона",
    "уровень и качество жизни населения",
    "миграция населения и рынок труда",
    "занятость и заработная плата",
    "жилищные условия и инфраструктура",
    "образование и здравоохранение в регионе",
    "демографическая ситуация в регионе"
]

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer(
    "paraphrase-multilingual-MiniLM-L12-v2",
    device=device
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
anchor_embeddings = model.encode(
    anchor_texts,
    normalize_embeddings=True,
    batch_size=8
)

In [ ]:
def row_iterator(df):
    for _, row in df.iterrows():
        yield {
            "text": row["news"],
            "region": row["region"],
            "date": row["date"]
        }

In [ ]:
from tqdm import tqdm
import numpy as np

def semantic_filter_stream(
    rows_iter,
    model,
    anchor_embeddings,
    threshold=0.34,
    batch_size=512,
    total=None
):
    buffer = []

    for row in tqdm(rows_iter, total=total, desc="Semantic filtering"):
        buffer.append(row)

        if len(buffer) == batch_size:
            texts = [r["text"] for r in buffer]

            emb = model.encode(
                texts,
                normalize_embeddings=True,
                batch_size=batch_size,
                show_progress_bar=False
            )

            sims = (emb @ anchor_embeddings.T).max(axis=1)

            for r, s in zip(buffer, sims):
                if s >= threshold:
                    yield r

            buffer.clear()

    # обработка хвоста
    if buffer:
        texts = [r["text"] for r in buffer]
        emb = model.encode(texts, normalize_embeddings=True)
        sims = (emb @ anchor_embeddings.T).max(axis=1)

        for r, s in zip(buffer, sims):
            if s >= threshold:
                yield r


In [ ]:
import csv

with open("filtered_news.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["date", "region", "text"]
    )
    writer.writeheader()

    for row in semantic_filter_stream(
        rows_iter=row_iterator(df_full_news),
        model=model,
        anchor_embeddings=anchor_embeddings,
        threshold=0.34,
        batch_size=512,
        total=len(df_full_news)
    ):
        writer.writerow({
            "date": row["date"],
            "region": row["region"],
            "text": row["text"]
        })


Semantic filtering: 100%|██████████| 2757969/2757969 [55:52<00:00, 822.64it/s]


In [ ]:
import pandas as pd

df_fil = pd.read_csv('filtered_news.csv')
df_fil

,date,region,text
0,2022-01-01,belgorod,Вячеслав Гладков: к строительству детских площ...
1,2022-01-01,belgorod,В правительстве Белгородской области оценили э...
2,2022-01-01,belgorod,В 2022 году поддержку по соцконтракту получат ...
3,2022-01-01,belgorod,Уважаемые жители села Шеино!
4,2022-01-01,belgorod,Уважаемые жители села Шеино!
...,...,...,...
544278,2022-12-31,grozniy,Рамзан Кадыров об обстановке в Грозном: Все ск...
544279,2022-12-31,grozniy,Молодогвардейцы проверили цены на проживание в...
544280,2022-12-31,grozniy,Уходящий год для Кавказа стал Годом сплочения ...
544281,2022-12-31,grozniy,В Минобрнауки ЧР подвели итоги освещения нацпр...
